In [ ]:
#pip install lyricsgenius

In [1]:
# Import dependencies
import lyricsgenius
from config import genius_token
import pandas as pd
import numpy as np
import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password
import time

In [9]:
genius = lyricsgenius.Genius(genius_token)

In [6]:
# Create DataFrame from Spotify API calls
song_genres_df = pd.read_csv('../Data/song_genres.csv')
song_genres_df = song_genres_df.dropna()
song_genres_df.head()

,song,song_id,artist,artist_id,category,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.001790,0.1450,0.529,81.112,214707.0,4.0
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.000000,0.3660,0.756,179.954,206046.0,4.0
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.000000,0.1340,0.467,93.005,161385.0,4.0
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,1.0,0.0944,0.4400,0.000007,0.0921,0.531,80.870,238805.0,4.0
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,1.0,0.0373,0.3180,0.000000,0.4140,0.329,140.013,185850.0,4.0


In [10]:
# Create a list of lyrics for the first 1917 songs
t0 = time.time()
lyrics_list = []
song_count = len(song_genres_df)
for i in range(int(song_count/500)+1):
    start = i*500
    end = (i+1)*500
    print(f'Finding lyrics for songs {start}-{end}:')
    for index, row in song_genres_df[start:end].iterrows():
        song = row['song']
        artist = row['artist']
        results = genius.search_songs(f'{song} {artist}')
        try:
            found_song = False
            for id, hit in enumerate(results['hits']):
                artist_name = hit['result']['primary_artist']['name'].lower()
                if row['artist'].lower() in artist_name:
                    found_song = True
                    song_id = hit['result']['id']
                    lyrics = genius.lyrics(urlthing=song_id)
                    lyrics_list.append(lyrics)
                    break
            if found_song == False:
                lyrics_list.append(None)
                print(index, 'Song not found.')
        except:
            lyrics_list.append(None)
            print(index, 'No result found.')
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'Lyrics added for {len(lyrics_list)} songs.')

Finding lyrics for songs 0-500:
7 Song not found.
15 Song not found.
23 Song not found.
24 Song not found.
45 Song not found.
60 Song not found.
66 Song not found.
86 Song not found.
112 Song not found.
120 Song not found.
145 Song not found.
148 Song not found.
152 Song not found.
159 Song not found.
162 Song not found.
166 Song not found.
167 Song not found.
169 Song not found.
171 Song not found.
174 Song not found.
180 Song not found.
182 Song not found.
189 Song not found.
193 Song not found.
197 Song not found.
199 Song not found.
203 Song not found.
210 Song not found.
211 Song not found.
213 Song not found.
217 Song not found.
228 Song not found.
229 Song not found.
233 Song not found.
293 Song not found.
302 Song not found.
306 Song not found.
338 Song not found.
339 Song not found.
346 Song not found.
348 Song not found.
349 Song not found.
364 Song not found.
371 Song not found.
372 Song not found.
378 Song not found.
381 Song not found.
383 Song not found.
387 Song not foun

ConnectionError: HTTPSConnectionPool(host='api.genius.com', port=443): Max retries exceeded with url: /search?q=Kings+of+Metal+Manowar (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11725aba8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [25]:
# Add lyrics of the rest of the songs
t0 = time.time()
song_count = len(song_genres_df[1917:])
for i in range(int(song_count/500)+1):
    start = i*500 + 1917
    end = (i+1)*500 + 1917
    print(f'Finding lyrics for songs {start}-{end}:')
    for index, row in song_genres_df[start:end].iterrows():
        song = row['song']
        artist = row['artist']
        results = genius.search_songs(f'{song} {artist}')
        try:
            found_song = False
            for id, hit in enumerate(results['hits']):
                artist_name = hit['result']['primary_artist']['name'].lower()
                if row['artist'].lower() in artist_name:
                    found_song = True
                    song_id = hit['result']['id']
                    lyrics = genius.lyrics(urlthing=song_id)
                    lyrics_list.append(lyrics)
                    break
            if found_song == False:
                lyrics_list.append(None)
                print(index, 'Song not found.')
        except:
            lyrics_list.append(None)
            print(index, 'No result found.')
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'Lyrics added for {len(lyrics_list)} songs.')

Finding lyrics for songs 1916-2416:
1936 Song not found.
1938 Song not found.
1939 Song not found.
1941 Song not found.
1948 Song not found.
1951 Song not found.
1952 Song not found.
1953 Song not found.
1955 Song not found.
1958 Song not found.
1959 Song not found.
1960 Song not found.
1962 Song not found.
1963 Song not found.
1964 Song not found.
1965 Song not found.
1967 Song not found.
1969 Song not found.
1973 Song not found.
1974 Song not found.
1975 Song not found.
1978 Song not found.
1983 Song not found.
1986 Song not found.
1987 Song not found.
1988 Song not found.
1989 Song not found.
1990 Song not found.
1993 Song not found.
1995 Song not found.
1996 Song not found.
1997 Song not found.
1998 Song not found.
1999 Song not found.
2000 Song not found.
2002 Song not found.
2004 Song not found.
2006 Song not found.
2007 Song not found.
2008 Song not found.
2009 Song not found.
2011 Song not found.
2012 Song not found.
2013 Song not found.
2014 Song not found.
2015 Song not found

3549 Song not found.
3561 Song not found.
3571 Song not found.
3574 Song not found.
3575 Song not found.
3580 Song not found.
3583 Song not found.
3595 Song not found.
3597 Song not found.
3598 Song not found.
3607 Song not found.
3612 Song not found.
3615 Song not found.
3621 Song not found.
3632 Song not found.
3634 Song not found.
3635 Song not found.
3641 Song not found.
3643 Song not found.
3650 Song not found.
3651 Song not found.
3652 Song not found.
3653 Song not found.
3660 Song not found.
3661 Song not found.
3662 Song not found.
3663 Song not found.
3665 Song not found.
3666 Song not found.
3667 Song not found.
3668 Song not found.
3671 Song not found.
3676 Song not found.
3678 Song not found.
3688 Song not found.
3693 Song not found.
3699 Song not found.
3701 Song not found.
3703 Song not found.
3707 Song not found.
3711 Song not found.
3714 Song not found.
3719 Song not found.
3722 Song not found.
3725 Song not found.
3728 Song not found.
3733 Song not found.
3736 Song not

4756 Song not found.
4757 Song not found.
4758 Song not found.
4759 Song not found.
4760 Song not found.
4763 Song not found.
4764 Song not found.
4765 Song not found.
4766 Song not found.
4767 Song not found.
4768 Song not found.
4770 Song not found.
4771 Song not found.
4772 Song not found.
4773 Song not found.
4774 Song not found.
4775 Song not found.
4776 Song not found.
4778 Song not found.
4779 Song not found.
4781 Song not found.
4782 Song not found.
4783 Song not found.
4784 Song not found.
4786 Song not found.
4787 Song not found.
4788 Song not found.
4789 Song not found.
4790 Song not found.
4791 Song not found.
4792 Song not found.
4793 Song not found.
4794 Song not found.
4795 Song not found.
4796 Song not found.
4797 Song not found.
4798 Song not found.
4799 Song not found.
4800 Song not found.
4801 Song not found.
4802 Song not found.
4803 Song not found.
4804 Song not found.
4805 Song not found.
4806 Song not found.
4807 Song not found.
4808 Song not found.
4809 Song not

5237 Song not found.
5239 Song not found.
5241 Song not found.
5243 Song not found.
5244 Song not found.
5246 Song not found.
5247 Song not found.
5249 Song not found.
5251 Song not found.
5252 Song not found.
5253 Song not found.
5256 Song not found.
5257 Song not found.
5258 Song not found.
5261 Song not found.
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
5262 No result found.
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
5265 No result found.
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
5269 No result found.
5271 Song not found.
5274 Song not found.
5275 Song not found.
5276 Song not found.
5277 Song not found.
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
5278 No result found.
5279 Song not found.
Timeout raised 

5763 Song not found.
5766 Song not found.
5768 Song not found.
5770 Song not found.
5771 Song not found.
5773 Song not found.
5774 Song not found.
5775 Song not found.
5776 Song not found.
5777 Song not found.
5780 Song not found.
5781 Song not found.
5782 Song not found.
5783 Song not found.
5784 Song not found.
5785 Song not found.
5786 Song not found.
5787 Song not found.
5788 Song not found.
5789 Song not found.
5790 Song not found.
5791 Song not found.
5792 Song not found.
5793 Song not found.
5794 Song not found.
5795 Song not found.
5796 Song not found.
5798 Song not found.
5799 Song not found.
5800 Song not found.
5801 Song not found.
5802 Song not found.
5803 Song not found.
5804 Song not found.
5805 Song not found.
5806 Song not found.
5807 Song not found.
5808 Song not found.
5809 Song not found.
5810 Song not found.
5811 Song not found.
5812 Song not found.
5813 Song not found.
5814 Song not found.
5815 Song not found.
5816 Song not found.
5818 Song not found.
5820 Song not

...

9988 Song not found.
9989 Song not found.
9990 Song not found.
9991 Song not found.
9992 Song not found.
9993 Song not found.
9994 Song not found.
9995 Song not found.
9996 Song not found.
9997 Song not found.
9998 Song not found.
9999 Song not found.
10001 Song not found.
10002 Song not found.
10003 Song not found.
10004 Song not found.
10005 Song not found.
10006 Song not found.
10007 Song not found.
10008 Song not found.
10009 Song not found.
10010 Song not found.
10011 Song not found.
10012 Song not found.
10013 Song not found.
10014 Song not found.
10015 Song not found.
10017 Song not found.
10018 Song not found.
10019 Song not found.
10020 Song not found.
10022 Song not found.
10023 Song not found.
10024 Song not found.
10025 Song not found.
10026 Song not found.
10027 Song not found.
10028 Song not found.
10030 Song not found.
10031 Song not found.
10032 Song not found.
10033 Song not found.
10034 Song not found.
10035 Song not found.
10036 Song not found.
10038 Song not found.


10526 Song not found.
10527 Song not found.
10528 Song not found.
10530 Song not found.
10532 Song not found.
10534 Song not found.
10536 Song not found.
10537 Song not found.
10538 Song not found.
10539 Song not found.
10541 Song not found.
10543 Song not found.
10546 Song not found.
10547 Song not found.
10548 Song not found.
10550 Song not found.
10553 Song not found.
10555 Song not found.
10556 Song not found.
10557 Song not found.
10559 Song not found.
10562 Song not found.
10570 Song not found.
10574 Song not found.
10578 Song not found.
10581 Song not found.
10585 Song not found.
10586 Song not found.
10588 Song not found.
10591 Song not found.
10596 Song not found.
10597 Song not found.
10598 Song not found.
10599 Song not found.
10603 Song not found.
10604 Song not found.
10607 Song not found.
10610 Song not found.
10611 Song not found.
10613 Song not found.
10617 Song not found.
10620 Song not found.
10621 Song not found.
10623 Song not found.
10624 Song not found.
10625 Song

11231 Song not found.
11232 Song not found.
11233 Song not found.
11234 Song not found.
11237 Song not found.
11238 Song not found.
11239 Song not found.
11240 Song not found.
11241 Song not found.
11242 Song not found.
11244 Song not found.
11245 Song not found.
11246 Song not found.
11247 Song not found.
11250 Song not found.
11251 Song not found.
11253 Song not found.
11254 Song not found.
11255 Song not found.
11259 Song not found.
11261 Song not found.
11262 Song not found.
11263 Song not found.
11265 Song not found.
11270 Song not found.
11273 Song not found.
11274 Song not found.
11275 Song not found.
11276 Song not found.
11277 Song not found.
11278 Song not found.
11279 Song not found.
11280 Song not found.
11281 Song not found.
11282 Song not found.
11284 Song not found.
11290 Song not found.
11292 Song not found.
11294 Song not found.
11298 Song not found.
11299 Song not found.
11300 Song not found.
11301 Song not found.
11303 Song not found.
11308 Song not found.
11314 Song

11890 Song not found.
11892 Song not found.
11895 Song not found.
11899 Song not found.
11902 Song not found.
11904 Song not found.
11906 Song not found.
11907 Song not found.
11908 Song not found.
11913 Song not found.
11915 Song not found.
11916 Song not found.
Finding lyrics for songs 11916-12416:
11921 Song not found.
11922 Song not found.
11923 Song not found.
11927 Song not found.
11928 Song not found.
11931 Song not found.
11932 Song not found.
11933 Song not found.
11935 Song not found.
11936 Song not found.
11939 Song not found.
11941 Song not found.
11943 Song not found.
11945 Song not found.
11946 Song not found.
11947 Song not found.
11948 Song not found.
11956 Song not found.
11963 Song not found.
11964 Song not found.
11965 Song not found.
11967 Song not found.
11970 Song not found.
11972 Song not found.
11973 Song not found.
11975 Song not found.
11976 Song not found.
11977 Song not found.
11978 Song not found.
11979 Song not found.
12014 Song not found.
12030 Song not f

14043 Song not found.
14044 Song not found.
14045 Song not found.
14047 Song not found.
14049 Song not found.
14050 Song not found.
14054 Song not found.
14055 Song not found.
14056 Song not found.
14061 Song not found.
14062 Song not found.
14063 Song not found.
14064 Song not found.
14065 Song not found.
14069 Song not found.
14070 Song not found.
14071 Song not found.
14072 Song not found.
14073 Song not found.
14074 Song not found.
14075 Song not found.
14077 Song not found.
14078 Song not found.
14079 Song not found.
14083 Song not found.
14084 Song not found.
14085 Song not found.
14089 Song not found.
14091 Song not found.
14092 Song not found.
14093 Song not found.
14096 Song not found.
14100 Song not found.
14101 Song not found.
14102 Song not found.
14106 Song not found.
14107 Song not found.
14109 Song not found.
14113 Song not found.
14114 Song not found.
14115 Song not found.
14119 Song not found.
14121 Song not found.
14122 Song not found.
14123 Song not found.
14126 Song

In [41]:
# Look for lyrics for HTTPSConnectionPool errors
https_errors = [932, 1090, 1455, 1917, 2877, 2884, 3011, 3012, 3247, 3381, 3816, 3943, 
                3947, 5262, 5265, 5269, 5278, 5280, 6241, 7636, 8139, 8726, 1471]
https_lyrics = []
for num in https_errors:
    for index, row in song_genres_df[num:num+1].iterrows():
        song = row['song']
        artist = row['artist']
        results = genius.search_songs(f'{song} {artist}')
        try:
            found_song = False
            for id, hit in enumerate(results['hits']):
                artist_name = hit['result']['primary_artist']['name'].lower()
                if row['artist'].lower() in artist_name:
                    found_song = True
                    song_id = hit['result']['id']
                    lyrics = genius.lyrics(urlthing=song_id)
                    https_lyrics.append(lyrics)
                    break
            if found_song == False:
                https_lyrics.append(None)
                print(index, 'Song not found.')
        except(Exception) as e:
            https_lyrics.append(None)
            print(index, 'No result found.')
print(len(https_errors))
len(https_lyrics)

1456 Song not found.
2885 Song not found.
3817 Song not found.
5279 Song not found.
8727 Song not found.
23


23

In [44]:
# Add HTTPSConnectionPool error lyrics to lyrics_list
for i in range(len(https_errors)):
    lyrics_list[https_errors[i]] = https_lyrics[i]

In [45]:
# Get the number of songs that without lyrics
lyrics_list.count(None)

5229

In [58]:
# Add a column of lyrics to the DataFrame
song_genres_df['lyrics'] = lyrics_list
song_genres_df.head()

,song,song_id,artist,artist_id,category,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.001790,0.1450,0.529,81.112,214707.0,4.0,[Verse 1]\nI'm like the water when your ship r...
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.000000,0.3660,0.756,179.954,206046.0,4.0,[Verse 1: Quinn XCII]\nDidn't even wanna go ou...
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.000000,0.1340,0.467,93.005,161385.0,4.0,"[Chorus]\nYou cut out a piece of me, and now I..."
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,1.0,0.0944,0.4400,0.000007,0.0921,0.531,80.870,238805.0,4.0,"[Intro]\n(Last night, all I think about is you..."
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,1.0,0.0373,0.3180,0.000000,0.4140,0.329,140.013,185850.0,4.0,[Chrorus]\nAre you okay?\n'Cause you're the on...


In [59]:
# Create a the clean_lyrics function
def clean_lyrics(song_genres_df):
    # Create non-destructive copy
    song_genres_df = pd.DataFrame(song_genres_df)
    
    # Create lists of symbols to clean
    regex = '\[(.*?)\]'
    symbols_to_space = ['\u205f', '\\u2005', '  ', '  ', '\n', '-', '–', '—', '_', '=']
    symbols_to_remove = [',', '.', '(', ')', "'", '’', '`', '[', ']', '{', '}', '"',
                         '”', '“', ':', ';', '*', '¨', '…', '‘', '·']
    other_symbols = ['?', '!'] 
    e = 'е'
    
    # Clean lyrics
    song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(regex, '', regex=True)
    for symbol in symbols_to_space:
        song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(symbol, ' ')
    for symbol in symbols_to_remove:
        song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(symbol, '')
    for symbol in other_symbols:
        song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(symbol, f' {symbol}')
    song_genres_df['lyrics'] = song_genres_df['lyrics'].str.replace(e, 'e')
    
    return song_genres_df

In [60]:
# Run the clean_lyrics function
song_lyrics_df = clean_lyrics(song_genres_df)
song_lyrics_df.head(3)

,song,song_id,artist,artist_id,category,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.001790,0.1450,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.000000,0.3660,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.000000,0.1340,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,1.0,0.0944,0.4400,0.000007,0.0921,0.531,80.870,238805.0,4.0,Last night all I think about is you Dont stop...
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,1.0,0.0373,0.3180,0.000000,0.4140,0.329,140.013,185850.0,4.0,Are you okay ? Cause youre the one who needed...


In [69]:
# Count non alpha words
non_alpha_words = []
for index, row in song_lyrics_df.iterrows():
    filtered_words = row['lyrics']
    count = 0
    for word in list(set(str(filtered_words).strip().split(' '))):
        if not re.match(r'[a-zA-Z0-9?!/#$&+]+$', word):
            count+=1
    non_alpha_words.append(count)
print(f'{len(non_alpha_words)} non alphabetic words found.')

14211 non alphabetic words found.


In [70]:
# Add a column for alpha word count
song_lyrics_df['non_alpha_words'] = non_alpha_words


In [119]:
## Print non alpha words for each song
#for item, row in song_lyrics_df[song_lyrics_df['non_alpha_words']>10].iterrows():
#    filtered_words = row['lyrics']
#    for word in list(set(str(filtered_words).strip().split(' '))):
#        if not re.match(r'[a-zA-Z0-9?!\/#$&+]+$', word):
#            print(row['song'], word)

In [120]:
# Remove songs with more than 10 non alpha words
#song_lyrics_df = song_lyrics_df[song_lyrics_df['non_alpha_words']<=10]
#song_lyrics_df.head()


,song,song_id,artist,artist_id,category,genres,filtered_genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,...,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"['pop', 'viral', 'canadian', 'post-teen', 'dan...","['pop', 'viral', 'canadian', 'dance']",0.652,0.383,2,...,0.0516,0.06760,0.00,0.0828,0.549,145.765,178994,4,You put me on a pedestal and tell me Im the b...,1
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"['pop', 'electropop']",['pop'],0.889,0.340,11,...,0.0697,0.21800,0.13,0.0550,0.716,94.009,174321,4,Im not your friend Or anything damn You think...,1
2,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"['pop', 'uk', 'dance']","['pop', 'uk', 'dance']",0.702,0.825,6,...,0.0601,0.00883,0.00,0.0674,0.915,102.977,203064,4,None,0
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"['pop', 'post-teen']",['pop'],0.736,0.802,0,...,0.0864,0.46800,0.00,0.0940,0.675,144.005,172325,4,Heaven sent you to me Im just hopin I dont re...,1
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"['lgbtq+', 'pop', 'queer', 'country']","['pop', 'country']",0.810,0.511,5,...,0.1640,0.12000,0.00,0.0832,0.837,151.947,154998,4,T T Tay Keith Took it to ten Hey Ayy its a ...,1


In [74]:
# Find the number of songs without lyrics
song_lyrics_df['lyrics'].isnull().sum()

5229

In [78]:
# Remove the songs without lyrics
song_lyrics_df = song_lyrics_df.dropna()
song_lyrics_df.head()

,song,song_id,artist,artist_id,category,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,...,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,"['dance', 'pop']",0.392,0.574,7.0,-9.195,...,0.1700,0.8330,0.001790,0.1450,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,...,0.2840,0.0805,0.000000,0.3660,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,['australian'],0.662,0.413,0.0,-7.357,...,0.0299,0.2130,0.000000,0.1340,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,...,0.0944,0.4400,0.000007,0.0921,0.531,80.870,238805.0,4.0,Last night all I think about is you Dont stop...,1
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,...,0.0373,0.3180,0.000000,0.4140,0.329,140.013,185850.0,4.0,Are you okay ? Cause youre the one who needed...,1


In [79]:
# Print the number of songs in the DataFrame
len(song_lyrics_df)

8982

In [77]:
# Save DataFrame to CSV
song_lyrics_df.to_csv('../Data/song_lyrics.csv', index=False)